In [1]:
from sklearn import datasets
import seaborn as sns
import pandas as pd
import numpy as np

In [2]:
pd.DataFrame(sns.get_dataset_names()).T

/home/rohan/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/rohan/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,anscombe,attention,brain_networks,car_crashes,diamonds,dots,exercise,flights,fmri,gammas,iris,mpg,planets,tips,titanic


In [3]:
data = sns.load_dataset('titanic')

In [4]:
data.shape

(891, 15)

In [10]:
data_f = data._get_numeric_data()

print(data_f.shape)
data_f.head(2)

(891, 8)


,survived,pclass,age,sibsp,parch,fare,adult_male,alone
0,0,3,22.0,1,0,7.2500,True,False
1,1,1,38.0,1,0,71.2833,False,False


In [6]:
data_f.survived.value_counts()

0    549
1    342
Name: survived, dtype: int64

In [12]:
survive = data_f[data_f.survived == 1]
died = data_f[data_f.survived == 0]

print(survive.shape)
print(died.shape)

(342, 8)
(549, 8)


In [14]:
data_f = pd.concat([survive.sample(200),died.sample(5400,replace= True)])
data_f.survived.value_counts()

0    5400
1     200
Name: survived, dtype: int64

#### Random Sampling

- df.sample gives you random samples from a series

- replace = with/without replacement

In [ ]:
x

In [22]:
np.random.seed(123)
x  = data_f.sample(200)

x.survived.value_counts()

0    195
1      5
Name: survived, dtype: int64

#### Sampling using Sklearn.Imbalance - Undersampling

In [24]:
data_f.age.fillna(method='bfill',inplace= True)

In [27]:
data_f.isnull().sum()

survived      0
pclass        0
age           0
sibsp         0
parch         0
fare          0
adult_male    0
alone         0
dtype: int64

In [29]:
X =  data_f[data_f.columns.difference(['survived'])]
Y = data_f.survived

In [30]:
X.shape,Y.shape

((5600, 7), (5600,))

In [31]:
Y.value_counts()

0    5400
1     200
Name: survived, dtype: int64

In [32]:
from imblearn.under_sampling import RandomUnderSampler

- Randomoversampler can hadle categorical var as well unless its the target variable unlike below mentioned

In [33]:
rus = RandomUnderSampler(sampling_strategy='majority',return_indices=False)

In [34]:
x_res, y_res = rus.fit_sample(X = X,y = Y)

- Here the minority count was only 200, so without losing info there,here we have reduced majority

In [35]:
pd.Series(y_res).value_counts()

1    200
0    200
dtype: int64

In [143]:
#id_rus - indicies of selected data

- similarly oversampling can be done

#### TomekLinks - For Undersampling

-Tomek links are pairs of very close instances, but of opposite classes. Removing the instances of the majority class of each pair increases the space between the two classes, facilitating the classification process.

In [36]:
from imblearn.under_sampling import TomekLinks

In [37]:
X.shape,Y.shape

((5600, 7), (5600,))

In [38]:
Y.value_counts()

0    5400
1     200
Name: survived, dtype: int64

- We can't use ratio = float for Tomeklinks unlike above used

In [39]:
tom = TomekLinks(return_indices=True,ratio= {0: 100})

In [40]:
x_tom,y_tom,A = tom.fit_sample(X,Y)

In [41]:
A

array([   0,    1,    2, ..., 5597, 5598, 5599])

In [42]:
x_tom.shape,y_tom.shape

((5583, 7), (5583,))

In [43]:
pd.Series(y_tom).value_counts()

0    5383
1     200
dtype: int64

#### Cluster Centroid

In [44]:
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(ratio={0: 100})
X_cc, y_cc = cc.fit_sample(X, Y)

In [45]:
pd.Series(y_cc).value_counts()

1    200
0    100
dtype: int64

#### Smote

In [46]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(ratio='minority')
X_sm, y_sm = smote.fit_sample(X, Y)


In [47]:
pd.Series(y_sm).value_counts()

1    5400
0    5400
dtype: int64

##### ----------------------------------------------

In [469]:
smote = SMOTE(ratio={1:3000},)
X_sm, y_sm = smote.fit_sample(X, Y)

In [476]:
pd.Series(y_sm).value_counts()

0    5400
1    3000
dtype: int64

#### combination of both - SMOTETomek

In [481]:
from imblearn.combine import SMOTETomek


smt = SMOTETomek(ratio={1:3000})
X_smt, y_smt = smt.fit_sample(X, Y)

In [475]:
pd.Series(y_smt).value_counts()

0    5375
1    2975
dtype: int64

In [480]:
y_smt

array([1, 1, 1, ..., 1, 1, 1])

#### Other Smote variants - [ SMOTENC : handles categorical var] || [ borderlineSmote : variant of Smote]

from imblearn.over_sampling import SMOTENC 

- variation of Smote which can handle categorical values as well

In [485]:
from imblearn.over_sampling import BorderlineSMOTE

smt = BorderlineSMOTE()
X_smt, y_smt = smt.fit_sample(X, Y)